## 4-3) Queue Runner

file 용량이 커서 한번에 메모리에 올리지 못할 경우 사용. 

크게 세 단계(filename list만들기, reader 설정, record default 설정)로 이뤄지나 세부적으로 4단계로 정리해봄.

1) 여러 개의 file에서 data를 읽어와야 하는 경우 filename_queue에 file 쌓기

2) reader로 연결(data 읽어오기)

3) 양식에 맞는 datatype으로 decode 
    
    ex: csv file을 예로 들면 구분자에 따라 데이터(숫자, 문자)를 분리하고 구분자는 제거  

4) Example queue에 data 쌓기: batch만큼 읽어와서 example queue에 쌓고 학습

In [1]:
import tensorflow as tf

/Users/beomyongnho/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
# filename_queue를 list 형태로 만들기
filename_queue = tf.train.string_input_producer(
    ['./data/X_dataset.csv'], shuffle=False, name='filename_queue')

# reader 정의 
reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# decode 양식 설정(record_defaults는 데이터가 비어있는 경우 기본값이 된다)
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

# batch size 및 어느 데이터가 x(data), y(label)가 될지 설정
train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)
    
# 행렬곱을 고려해서 matrix shape 설정할 것!
X = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.placeholder(tf.float32, shape=[None, 1])
W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')
yhat = tf.matmul(X, W) + b

cost = tf.reduce_mean(tf.square(yhat - y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-05)
train_op = optimizer.minimize(cost)

In [12]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for i in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    _, _cost, _yhat = sess.run([train_op, cost, yhat], 
                              feed_dict={X: x_batch, 
                                         y: y_batch})
    if i%100==0:
        print('step: {}, cost: {}, yhat: {}'
              .format(i, _cost, _yhat))

coord.request_stop()
coord.join(threads)


step: 0, cost: 55927.6171875, yhat: [[-58.74794769]
 [-67.29358673]
 [-67.87072754]
 [-74.56163025]
 [-50.16615295]
 [-58.74794769]
 [-67.29358673]
 [-67.87072754]
 [-74.56163025]
 [-50.16615295]]
step: 100, cost: 0.25394362211227417, yhat: [[ 151.09890747]
 [ 184.90914917]
 [ 180.63807678]
 [ 196.05688477]
 [ 142.19778442]
 [ 151.09890747]
 [ 184.90914917]
 [ 180.63807678]
 [ 196.05688477]
 [ 142.19778442]]
step: 200, cost: 0.25107890367507935, yhat: [[ 151.10795593]
 [ 184.9029541 ]
 [ 180.6408844 ]
 [ 196.05860901]
 [ 142.18991089]
 [ 151.10795593]
 [ 184.9029541 ]
 [ 180.6408844 ]
 [ 196.05860901]
 [ 142.18991089]]
step: 300, cost: 0.2483580857515335, yhat: [[ 151.11677551]
 [ 184.89692688]
 [ 180.64360046]
 [ 196.06031799]
 [ 142.18226624]
 [ 151.11677551]
 [ 184.89692688]
 [ 180.64360046]
 [ 196.06031799]
 [ 142.18226624]]
step: 400, cost: 0.2457858771085739, yhat: [[ 151.12536621]
 [ 184.89109802]
 [ 180.64628601]
 [ 196.06195068]
 [ 142.17481995]
 [ 151.12536621]
 [ 184.8910980